In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
import torchvision.transforms as transforms
import torchvision.models as models
from torch import optim
import json
import torch.optim.lr_scheduler as lr_scheduler
import os

from tqdm import tqdm
from utils import util_functions, metric_functions, custom_dataset

In [27]:
train_labels_path = os.path.join('dataset', 'train.csv')
test_labels_path = os.path.join('dataset', 'test.csv')

train_images_dir = os.path.join('dataset', 'train')
test_images_dir = os.path.join('dataset', 'test')

train_labels_df = pd.read_csv(train_labels_path)
test_labels_df = pd.read_csv(test_labels_path)

In [44]:
img_size = 400
with_path = False
# Calculate the mean and std of dataset
base_transformer =  transforms.Compose([transforms.Resize((img_size, img_size)),
                                transforms.ToTensor()])

base_test_dataset = custom_dataset.CustomDataset(images_dir=test_images_dir, 
                                                df=train_labels_df, transforms=base_transformer, is_test=True)

test_mean, test_std = util_functions.calculate_mean_std_of_dataset(base_test_dataset, is_test=True)
test_transformer = transforms.Compose([transforms.Resize((img_size, img_size)),
                                transforms.ToTensor(),
                                transforms.Normalize(test_mean, test_std)])

test_dataset = custom_dataset.CustomDataset(images_dir=test_images_dir, 
                                                df=train_labels_df, transforms=test_transformer, is_test=True)

## Checking how many different classes we have
no_classes = len(list(train_labels_df['label'].unique()))
BATCH_SIZE = 64
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

tensor([0.4741, 0.4623, 0.4588])
tensor([0.2617, 0.2613, 0.2666])
6616 827 4080


In [ ]:
resnet34_model = models.resnet34(pretrained=True)
resnet34_model.fc.out_features = no_classes
resnet34_model.load_state_dict(torch.load('results/2023-12-01_13-18/best_model.pt', map_location='cpu'))

efficientnet_v2_m_model = models.efficientnet_v2_s(pretrained=True)
efficientnet_v2_m_model.classifier[-1].out_features = no_classes
efficientnet_v2_m_model.load_state_dict(torch.load('results/2023-12-01_14-55/best_model.pt', map_location='cpu'))

efficientnet_v2_s_model = models.efficientnet_v2_s(pretrained=True)
efficientnet_v2_s_model.classifier[-1].out_features = no_classes
efficientnet_v2_s_model.load_state_dict(torch.load('results/2023-12-01_14-55/best_model.pt', map_location='cpu'))

resnet152_model = models.resnet152(pretrained=True)
resnet152_model.fc.out_features = no_classes
resnet152_model.load_state_dict(torch.load('results/2023-12-01_16-16/best_model.pt', map_location='cpu'))

mobilenet_v2_model = models.mobilenet_v2(pretrained=True)
mobilenet_v2_model.classifier[-1].out_features = no_classes
mobilenet_v2_model.load_state_dict(torch.load('results/2023-12-01_17-05/best_model.pt', map_location='cpu'))

efficientnet_v2_s2_model = models.efficientnet_v2_s(pretrained=True)
efficientnet_v2_s2_model.classifier[-1].out_features = no_classes
efficientnet_v2_s2_model.load_state_dict(torch.load('results/2023-12-01_18-01/best_model.pt', map_location='cpu'))

resnet18_model = models.resnet34(pretrained=True)
resnet18_model.fc.out_features = no_classes
resnet18_model.load_state_dict(torch.load('results/2023-12-01_19-01/best_model.pt', map_location='cpu'))

densenet121_model = models.densenet121(pretrained=True)
densenet121_model.classifier.out_features = no_classes
densenet121_model.load_state_dict(torch.load('results/2023-12-01_19-38/best_model.pt', map_location='cpu'))

resnet34_model = models.resnet34(pretrained=True)
resnet34_model.fc.out_features = no_classes
resnet34_model.load_state_dict(torch.load('results/2023-12-01_22-54/best_model.pt', map_location='cpu'))

models_list = [resnet34_model, efficientnet_v2_s_model, resnet152_model, mobilenet_v2_model, efficientnet_v2_s2_model, resnet18_model, densenet121_model, resnet34_model]
from PIL import Image
for i, model in enumerate(models_list):
    predicted_labels = []
    model.eval()
    with torch.no_grad():
        for path in list(test_labels_df['id'].values):
            with open(f'dataset/{path}', 'rb') as f:
                img = Image.open(f).convert('RGB')
                img = test_transformer(img)
                img = img.unsqueeze(0)
                output = model(img)
                _, pred = torch.max(output.data, 1)
                predicted_labels.append(pred.item())
    test_labels_df['Predicted'] = predicted_labels
    save_results_path = f'dataset/final{i+1}.csv'
    test_labels_df.to_csv(save_results_path, index=False)

In [ ]:
df1 = pd.read_csv('dataset/final1.csv')
df2 = pd.read_csv('dataset/final2.csv')
df3 = pd.read_csv('dataset/final3.csv')
df4 = pd.read_csv('dataset/final4.csv')
df5 = pd.read_csv('dataset/final5.csv')
df6 = pd.read_csv('dataset/final6.csv')
df7 = pd.read_csv('dataset/final7.csv')
df8 = pd.read_csv('dataset/final8.csv')
df9 = pd.read_csv('dataset/final9.csv')

all_dfs = df1
for temp_df in [df2, df3, df4, df5, df6, df7, df8]:
    all_dfs = pd.concat([all_dfs, temp_df['Predicted']], axis=1)
    
import copy

majority_col1 = all_dfs.mode(axis=1, numeric_only=True).iloc[:, 0]
majority_col1 = [int(each) for each in majority_col1.values]
id_list = list(all_dfs['id'])
final_df = pd.DataFrame({'id':id_list, 'Predicted':majority_col1})

save_results_path = 'dataset/final10.csv'
final_df.to_csv(save_results_path, index=False)

In [138]:
final_df.head(20)

,id,Predicted
0,test/4136.jpg,92
1,test/4137.jpg,67
2,test/4138.jpg,21
3,test/4139.jpg,49
4,test/4140.jpg,38
5,test/4141.jpg,97
6,test/4142.jpg,72
7,test/4143.jpg,40
8,test/4144.jpg,22
9,test/4145.jpg,11


In [140]:
all_dfs

,id,Predicted,Predicted,Predicted,Predicted,Predicted,Predicted,Predicted,Predicted
0,test/4136.jpg,90,92,92,92,91,92,93,93
1,test/4137.jpg,67,67,67,67,67,67,67,67
2,test/4138.jpg,21,21,21,21,21,21,21,21
3,test/4139.jpg,49,49,49,49,49,49,49,49
4,test/4140.jpg,38,38,38,38,38,38,38,38
...,...,...,...,...,...,...,...,...,...
4075,test/8211.jpg,52,52,52,52,52,52,52,52
4076,test/8212.jpg,7,7,7,96,7,7,5,7
4077,test/8213.jpg,6,5,6,6,5,5,5,6
4078,test/8214.jpg,27,27,27,27,27,27,27,27
